# Install required libraries

In [ ]:
!pip install librosa soundfile opensmile speechbrain transformers torch openai-whisper
!pip install pandas numpy matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.0/996.0 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 

# Mount to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ADReSSoAnalyzer

In [ ]:
import os
import pandas as pd
import numpy as np
import librosa
import soundfile as sf
from pathlib import Path
import pickle
import json
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')
import opensmile
import torch
import whisper
from transformers import Wav2Vec2Processor, Wav2Vec2Model, BertTokenizer, BertModel
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class ADReSSoAnalyzer:
    def __init__(self, base_path="/content/drive/MyDrive/Voice/extracted/ADReSSo21"):
        self.base_path = base_path
        self.output_path = "/content"
        self.features = {}
        self.transcripts = {}

        # Initialize feature extractors
        self.smile = opensmile.Smile(
            feature_set=opensmile.FeatureSet.eGeMAPSv02,
            feature_level=opensmile.FeatureLevel.Functionals,
        )

        # Initialize Whisper for transcription
        self.whisper_model = whisper.load_model("base")

        # Initialize Wav2Vec2
        self.wav2vec_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        self.wav2vec_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

        # Initialize BERT
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')

    def get_audio_files(self) -> Dict[str, List[str]]:
        """Get all audio files from the dataset"""
        audio_files = {
            'diagnosis_ad': [],
            'diagnosis_cn': [],
            'progression_decline': [],
            'progression_no_decline': [],
            'progression_test': []
        }

        # Diagnosis files
        diag_ad_path = f"{self.base_path}/diagnosis/train/audio/ad"
        diag_cn_path = f"{self.base_path}/diagnosis/train/audio/cn"

        if os.path.exists(diag_ad_path):
            audio_files['diagnosis_ad'] = [f"{diag_ad_path}/{f}" for f in os.listdir(diag_ad_path) if f.endswith('.wav')]
        if os.path.exists(diag_cn_path):
            audio_files['diagnosis_cn'] = [f"{diag_cn_path}/{f}" for f in os.listdir(diag_cn_path) if f.endswith('.wav')]

        # Progression files
        prog_decline_path = f"{self.base_path}/progression/train/audio/decline"
        prog_no_decline_path = f"{self.base_path}/progression/train/audio/no_decline"
        prog_test_path = f"{self.base_path}/progression/test-dist/audio"

        if os.path.exists(prog_decline_path):
            audio_files['progression_decline'] = [f"{prog_decline_path}/{f}" for f in os.listdir(prog_decline_path) if f.endswith('.wav')]
        if os.path.exists(prog_no_decline_path):
            audio_files['progression_no_decline'] = [f"{prog_no_decline_path}/{f}" for f in os.listdir(prog_no_decline_path) if f.endswith('.wav')]
        if os.path.exists(prog_test_path):
            audio_files['progression_test'] = [f"{prog_test_path}/{f}" for f in os.listdir(prog_test_path) if f.endswith('.wav')]

        return audio_files

    def extract_acoustic_features(self, audio_path: str) -> Dict[str, Any]:
        """Extract all acoustic features from audio file"""
        features = {}

        try:
            # Load audio - resample to 16kHz for Wav2Vec2 compatibility
            y, sr = librosa.load(audio_path, sr=16000)  # Force 16kHz sampling rate

            # 1. eGeMAPS features using openSMILE
            try:
                features['egemaps'] = self.smile.process_file(audio_path).values.flatten()
            except Exception as e:
                print(f"  Warning: eGeMAPS extraction failed for {os.path.basename(audio_path)}: {str(e)}")
                features['egemaps'] = np.zeros(88)  # Default eGeMAPS feature size

            # 2. MFCC features
            try:
                mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
                features['mfccs'] = {
                    'mean': np.mean(mfccs, axis=1),
                    'std': np.std(mfccs, axis=1),
                    'delta': np.mean(librosa.feature.delta(mfccs), axis=1),
                    'delta2': np.mean(librosa.feature.delta(mfccs, order=2), axis=1)
                }
            except Exception as e:
                print(f"  Warning: MFCC extraction failed for {os.path.basename(audio_path)}: {str(e)}")
                features['mfccs'] = {
                    'mean': np.zeros(13),
                    'std': np.zeros(13),
                    'delta': np.zeros(13),
                    'delta2': np.zeros(13)
                }

            # 3. Log-mel spectrogram
            try:
                mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=80)
                log_mel = librosa.power_to_db(mel_spec)
                features['log_mel'] = {
                    'mean': np.mean(log_mel, axis=1),
                    'std': np.std(log_mel, axis=1)
                }
            except Exception as e:
                print(f"  Warning: Log-mel extraction failed for {os.path.basename(audio_path)}: {str(e)}")
                features['log_mel'] = {
                    'mean': np.zeros(80),
                    'std': np.zeros(80)
                }

            # 4. Wav2Vec2 features - with proper sampling rate handling
            try:
                # Ensure sampling rate is exactly 16000 Hz for Wav2Vec2
                if len(y) == 0:
                    raise ValueError("Empty audio signal")

                input_values = self.wav2vec_processor(
                    y,
                    sampling_rate=16000,  # Explicitly set to 16000
                    return_tensors="pt"
                ).input_values

                with torch.no_grad():
                    wav2vec_features = self.wav2vec_model(input_values).last_hidden_state
                features['wav2vec2'] = torch.mean(wav2vec_features, dim=1).squeeze().numpy()

            except Exception as e:
                print(f"  Warning: Wav2Vec2 extraction failed for {os.path.basename(audio_path)}: {str(e)}")
                features['wav2vec2'] = np.zeros(768)  # Default Wav2Vec2 feature size

            # 5. Additional prosodic features
            try:
                # Handle potential issues with F0 extraction
                f0 = librosa.yin(y, fmin=50, fmax=300, sr=sr)
                f0_clean = f0[f0 > 0]  # Remove unvoiced frames

                features['prosodic'] = {
                    'f0_mean': np.mean(f0_clean) if len(f0_clean) > 0 else 0.0,
                    'f0_std': np.std(f0_clean) if len(f0_clean) > 0 else 0.0,
                    'energy_mean': np.mean(librosa.feature.rms(y=y)),
                    'energy_std': np.std(librosa.feature.rms(y=y)),
                    'zero_crossing_rate': np.mean(librosa.feature.zero_crossing_rate(y)),
                    'spectral_centroid': np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
                    'spectral_rolloff': np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)),
                    'duration': len(y) / sr
                }
            except Exception as e:
                print(f"  Warning: Prosodic feature extraction failed for {os.path.basename(audio_path)}: {str(e)}")
                features['prosodic'] = {
                    'f0_mean': 0.0, 'f0_std': 0.0, 'energy_mean': 0.0, 'energy_std': 0.0,
                    'zero_crossing_rate': 0.0, 'spectral_centroid': 0.0, 'spectral_rolloff': 0.0,
                    'duration': 0.0
                }

        except Exception as e:
            print(f"Error processing {audio_path}: {str(e)}")
            features = None

        return features

    def show_acoustic_features(self, sample_file: str):
        """Display acoustic features for a sample file"""
        features = self.extract_acoustic_features(sample_file)

        if features is None:
            print(f"Could not extract features from {sample_file}")
            return

        print(f"=== Acoustic Features for {os.path.basename(sample_file)} ===\n")

        # eGeMAPS
        print(f"1. eGeMAPS Features: {len(features['egemaps'])} features")
        print(f"   Shape: {features['egemaps'].shape}")
        print(f"   Sample values: {features['egemaps'][:5]}")
        print()

        # MFCCs
        print("2. MFCC Features:")
        print(f"   Mean: {features['mfccs']['mean'].shape} - {features['mfccs']['mean'][:5]}")
        print(f"   Std: {features['mfccs']['std'].shape} - {features['mfccs']['std'][:5]}")
        print(f"   Delta: {features['mfccs']['delta'].shape} - {features['mfccs']['delta'][:5]}")
        print(f"   Delta-Delta: {features['mfccs']['delta2'].shape} - {features['mfccs']['delta2'][:5]}")
        print()

        # Log-mel
        print("3. Log-Mel Spectrogram Features:")
        print(f"   Mean: {features['log_mel']['mean'].shape} - {features['log_mel']['mean'][:5]}")
        print(f"   Std: {features['log_mel']['std'].shape} - {features['log_mel']['std'][:5]}")
        print()

        # Wav2Vec2
        print(f"4. Wav2Vec2 Features: {features['wav2vec2'].shape}")
        print(f"   Sample values: {features['wav2vec2'][:5]}")
        print()

        # Prosodic
        print("5. Prosodic Features:")
        for key, value in features['prosodic'].items():
            print(f"   {key}: {value:.4f}")
        print()

    def extract_transcripts(self, audio_files: Dict[str, List[str]]) -> Dict[str, str]:
        """Extract transcripts using Whisper"""
        transcripts = {}

        print("Extracting transcripts...")

        for category, files in audio_files.items():
            print(f"\nProcessing {category}...")
            for file_path in files:
                try:
                    filename = os.path.basename(file_path)
                    print(f"  Transcribing {filename}...")

                    result = self.whisper_model.transcribe(file_path)
                    transcript_text = result["text"].strip()

                    transcripts[f"{category}_{filename}"] = {
                        'file_path': file_path,
                        'category': category,
                        'filename': filename,
                        'transcript': transcript_text,
                        'language': result.get('language', 'en'),
                        'segments': len(result.get('segments', []))
                    }

                except Exception as e:
                    print(f"    Error transcribing {filename}: {str(e)}")
                    transcripts[f"{category}_{filename}"] = {
                        'file_path': file_path,
                        'category': category,
                        'filename': filename,
                        'transcript': "",
                        'error': str(e)
                    }

        return transcripts

    def save_transcripts(self, transcripts: Dict[str, str]):
        """Save transcripts to files"""
        os.makedirs(f"{self.output_path}/transcripts", exist_ok=True)

        # Save individual transcript files
        for key, data in transcripts.items():
            filename = f"{key}_transcript.txt"
            filepath = f"{self.output_path}/transcripts/{filename}"

            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(data['transcript'])

        # Save consolidated JSON
        with open(f"{self.output_path}/transcripts/all_transcripts.json", 'w', encoding='utf-8') as f:
            json.dump(transcripts, f, indent=2, ensure_ascii=False)

        # Save as pickle for easy loading
        with open(f"{self.output_path}/transcripts/transcripts.pkl", 'wb') as f:
            pickle.dump(transcripts, f)

        print(f"Transcripts saved to {self.output_path}/transcripts/")

    def create_transcript_table(self, transcripts: Dict[str, str]) -> pd.DataFrame:
        """Create a DataFrame with transcript information"""
        data = []

        for key, info in transcripts.items():
            data.append({
                'File_ID': key,
                'Category': info['category'],
                'Filename': info['filename'],
                'Transcript_Length': len(info['transcript']),
                'Word_Count': len(info['transcript'].split()) if info['transcript'] else 0,
                'Language': info.get('language', 'N/A'),
                'Segments': info.get('segments', 'N/A'),
                'Has_Error': 'error' in info,
                'Transcript_Preview': info['transcript'][:100] + "..." if len(info['transcript']) > 100 else info['transcript']
            })

        df = pd.DataFrame(data)

        # Save the table
        df.to_csv(f"{self.output_path}/transcript_summary.csv", index=False)

        return df

    def extract_linguistic_features(self, transcripts: Dict[str, str]) -> Dict[str, Any]:
        """Extract linguistic features for BERT preparation"""
        linguistic_features = {}

        print("Extracting linguistic features...")

        for key, data in transcripts.items():
            transcript = data['transcript']

            if not transcript:
                linguistic_features[key] = {
                    'raw_text': '',
                    'word_count': 0,
                    'sentence_count': 0,
                    'avg_word_length': 0,
                    'bert_tokens': [],
                    'bert_input_ids': [],
                    'bert_attention_mask': []
                }
                continue

            # Basic linguistic features
            words = transcript.split()
            sentences = transcript.split('.')

            # BERT tokenization
            bert_encoding = self.bert_tokenizer(
                transcript,
                truncation=True,
                padding='max_length',
                max_length=512,
                return_tensors='pt'
            )

            linguistic_features[key] = {
                'raw_text': transcript,
                'word_count': len(words),
                'sentence_count': len([s for s in sentences if s.strip()]),
                'avg_word_length': np.mean([len(word) for word in words]) if words else 0,
                'unique_words': len(set(words)),
                'lexical_diversity': len(set(words)) / len(words) if words else 0,
                'bert_tokens': self.bert_tokenizer.tokenize(transcript),
                'bert_input_ids': bert_encoding['input_ids'].squeeze().tolist(),
                'bert_attention_mask': bert_encoding['attention_mask'].squeeze().tolist(),
                'bert_encoding': bert_encoding
            }

        # Save linguistic features
        with open(f"{self.output_path}/linguistic_features.pkl", 'wb') as f:
            pickle.dump(linguistic_features, f)

        return linguistic_features

    def run_complete_pipeline(self):
        """Run the complete analysis pipeline"""
        print("=== ADReSSo21 Speech Analysis Pipeline ===\n")

        # Step 0: Get audio files
        print("Step 0: Getting audio files...")
        audio_files = self.get_audio_files()

        total_files = sum(len(files) for files in audio_files.values())
        print(f"Found {total_files} audio files across all categories")

        for category, files in audio_files.items():
            print(f"  {category}: {len(files)} files")

        if total_files == 0:
            print("No audio files found. Please check the dataset path.")
            return

        # Step 1: Show acoustic features for sample files
        print("\n" + "="*50)
        print("Step 1: Demonstrating acoustic features...")

        # Show features for one file from each category that has files
        for category, files in audio_files.items():
            if files:
                print(f"\nShowing features for {category}:")
                self.show_acoustic_features(files[0])
                break  # Just show one example to avoid too much output

        # Step 2: Extract transcripts
        print("\n" + "="*50)
        print("Step 2: Extracting transcripts...")
        transcripts = self.extract_transcripts(audio_files)

        # Step 3: Save transcripts
        print("\n" + "="*50)
        print("Step 3: Saving transcripts...")
        self.save_transcripts(transcripts)

        # Step 4: Create transcript table
        print("\n" + "="*50)
        print("Step 4: Creating transcript table...")
        transcript_df = self.create_transcript_table(transcripts)

        print("Transcript Summary Table:")
        print(transcript_df.to_string(index=False))

        # Step 5: Extract linguistic features for BERT
        print("\n" + "="*50)
        print("Step 5: Extracting linguistic features for BERT...")
        linguistic_features = self.extract_linguistic_features(transcripts)

        print("\nPipeline completed successfully!")
        print(f"Results saved to: {self.output_path}")
        print("\nOutput files:")
        print(f"  - Transcripts: {self.output_path}/transcripts/")
        print(f"  - Transcript summary: {self.output_path}/transcript_summary.csv")
        print(f"  - Linguistic features: {self.output_path}/linguistic_features.pkl")

        return {
            'audio_files': audio_files,
            'transcripts': transcripts,
            'transcript_df': transcript_df,
            'linguistic_features': linguistic_features
        }

# # Usage example
# if __name__ == "__main__":
#     # Initialize analyzer
#     analyzer = ADReSSoAnalyzer()

#     # Run complete pipeline
#     results = analyzer.run_complete_pipeline()

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.3 MB/s eta 0:00:00


# Architecture

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch_geometric
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool
from torch_geometric.data import Data, Batch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import networkx as nx
from transformers import BertModel
import pickle
import json
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

class GraphAttentionModule(nn.Module):
    """Graph-based attention module for semantic relationships"""
    def __init__(self, input_dim=768, hidden_dim=256, num_heads=8, num_layers=3):
        super(GraphAttentionModule, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads

        # Graph attention layers
        self.gat_layers = nn.ModuleList([
            GATConv(input_dim if i == 0 else hidden_dim,
                   hidden_dim, heads=num_heads, dropout=0.2)
            for i in range(num_layers)
        ])

        # Final projection
        self.projection = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, edge_index, batch=None):
        # Apply GAT layers
        for i, gat_layer in enumerate(self.gat_layers):
            x = gat_layer(x, edge_index)
            x = F.relu(x)
            x = self.dropout(x)

        # Global pooling if batch is provided
        if batch is not None:
            x = global_mean_pool(x, batch)
        else:
            x = torch.mean(x, dim=0, keepdim=True)

        return self.projection(x)

class VisionTransformerModule(nn.Module):
    """Vision Transformer for processing spectrograms"""
    def __init__(self, input_dim=80, patch_size=8, embed_dim=768, num_heads=12, num_layers=6):
        super(VisionTransformerModule, self).__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim

        # Patch embedding
        self.patch_embed = nn.Conv2d(1, embed_dim, kernel_size=patch_size, stride=patch_size)

        # Positional encoding
        self.pos_embed = nn.Parameter(torch.randn(1, 1000, embed_dim))  # Max patches

        # Transformer layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=num_heads, dim_feedforward=embed_dim*4,
            dropout=0.1, activation='gelu'
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Classification head
        self.classifier = nn.Linear(embed_dim, 256)

    def forward(self, x):
        # x shape: (batch_size, channels, height, width)
        B, C, H, W = x.shape

        # Create patches
        x = self.patch_embed(x)  # (B, embed_dim, H', W')
        x = x.flatten(2).transpose(1, 2)  # (B, num_patches, embed_dim)

        # Add positional encoding
        num_patches = x.shape[1]
        x = x + self.pos_embed[:, :num_patches, :]

        # Apply transformer
        x = x.transpose(0, 1)  # (num_patches, B, embed_dim)
        x = self.transformer(x)
        x = x.transpose(0, 1)  # (B, num_patches, embed_dim)

        # Global average pooling
        x = torch.mean(x, dim=1)  # (B, embed_dim)

        return self.classifier(x)

class UNetModule(nn.Module):
    """U-Net for audio feature processing"""
    def __init__(self, in_channels=1, out_channels=128):
        super(UNetModule, self).__init__()

        # Encoder
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)

        # Decoder
        self.dec4 = self.upconv_block(1024, 512)
        self.dec3 = self.upconv_block(512, 256)
        self.dec2 = self.upconv_block(256, 128)
        self.dec1 = self.upconv_block(128, 64)

        # Final layer
        self.final = nn.Conv1d(64, out_channels, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool1d(1)

    def conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True)
        )

    def upconv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.ConvTranspose1d(in_ch, out_ch, kernel_size=2, stride=2),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(F.max_pool1d(e1, 2))
        e3 = self.enc3(F.max_pool1d(e2, 2))
        e4 = self.enc4(F.max_pool1d(e3, 2))

        # Bottleneck
        b = self.bottleneck(F.max_pool1d(e4, 2))

        # Decoder
        d4 = self.dec4(b)
        d3 = self.dec3(d4)
        d2 = self.dec2(d3)
        d1 = self.dec1(d2)

        # Final
        out = self.final(d1)
        out = self.pool(out).squeeze(-1)  # Global average pooling

        return out

class AlexNetModule(nn.Module):
    """Modified AlexNet for feature extraction"""
    def __init__(self, input_dim=768, num_classes=256):
        super(AlexNetModule, self).__init__()

        self.features = nn.Sequential(
            nn.Linear(input_dim, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
        )

        self.classifier = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

class MultiModalADReSSoModel(nn.Module):
    """Complete multi-modal architecture"""
    def __init__(self,
                 audio_feature_dim=768,
                 text_feature_dim=768,
                 spectrogram_height=80,
                 num_classes=2):
        super(MultiModalADReSSoModel, self).__init__()

        # Initialize modules
        self.graph_attention = GraphAttentionModule(input_dim=text_feature_dim)
        self.vision_transformer = VisionTransformerModule(input_dim=spectrogram_height)
        self.unet = UNetModule()
        self.alexnet = AlexNetModule(input_dim=audio_feature_dim)

        # BERT for text processing
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Fusion layers
        self.fusion_layer = nn.Sequential(
            nn.Linear(256 + 256 + 128 + 256, 512),  # Graph + ViT + UNet + AlexNet
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Final classifier
        self.classifier = nn.Linear(256, num_classes)

    def create_semantic_graph(self, text_features, audio_features):
        """Create semantic relationship graph between audio and text"""
        batch_size = text_features.shape[0]
        graphs = []

        for i in range(batch_size):
            # Compute similarity matrix
            text_feat = text_features[i].unsqueeze(0)  # (1, dim)
            audio_feat = audio_features[i].unsqueeze(0)  # (1, dim)

            # Create nodes (text + audio features)
            node_features = torch.cat([text_feat, audio_feat], dim=0)  # (2, dim)

            # Create edges based on similarity
            similarity = F.cosine_similarity(text_feat, audio_feat, dim=1)

            # Create bidirectional edges if similarity > threshold
            if similarity.item() > 0.1:
                edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long).t()
            else:
                # Self-loops only
                edge_index = torch.tensor([[0, 1], [0, 1]], dtype=torch.long).t()

            graph = Data(x=node_features, edge_index=edge_index)
            graphs.append(graph)

        return Batch.from_data_list(graphs)

    def forward(self, audio_features, text_input_ids, text_attention_mask, spectrograms):
        batch_size = audio_features.shape[0]

        # Process text with BERT
        bert_outputs = self.bert(input_ids=text_input_ids, attention_mask=text_attention_mask)
        text_features = bert_outputs.last_hidden_state.mean(dim=1)  # (batch_size, 768)

        # Create semantic graph
        graph_batch = self.create_semantic_graph(text_features, audio_features)

        # Process through different modules
        graph_out = self.graph_attention(graph_batch.x, graph_batch.edge_index, graph_batch.batch)
        vit_out = self.vision_transformer(spectrograms)

        # Prepare audio for U-Net (add channel dimension)
        audio_1d = audio_features.unsqueeze(1)  # (batch_size, 1, features)
        unet_out = self.unet(audio_1d)

        alexnet_out = self.alexnet(audio_features)

        # Fusion
        fused_features = torch.cat([graph_out, vit_out, unet_out, alexnet_out], dim=1)
        fused_features = self.fusion_layer(fused_features)

        # Final classification
        output = self.classifier(fused_features)

        return output

class ADReSSoDataset(Dataset):
    """Dataset class for ADReSSo data"""
    def __init__(self, features_dict, linguistic_features, labels, transform=None):
        self.features_dict = features_dict
        self.linguistic_features = linguistic_features
        self.labels = labels
        self.transform = transform
        self.file_ids = list(features_dict.keys())

    def __len__(self):
        return len(self.file_ids)

    def __getitem__(self, idx):
        file_id = self.file_ids[idx]

        # Get features
        features = self.features_dict[file_id]
        linguistic = self.linguistic_features[file_id]

        # Prepare audio features (Wav2Vec2)
        audio_features = torch.FloatTensor(features['wav2vec2'])

        # Prepare text features
        text_input_ids = torch.LongTensor(linguistic['bert_input_ids'])
        text_attention_mask = torch.LongTensor(linguistic['bert_attention_mask'])

        # Prepare spectrogram (create from log-mel features)
        log_mel_mean = features['log_mel']['mean']
        log_mel_std = features['log_mel']['std']
        spectrogram = np.stack([log_mel_mean, log_mel_std])  # (2, 80)
        spectrogram = np.expand_dims(spectrogram.mean(axis=0), axis=0)  # (1, 80)
        spectrogram = np.tile(spectrogram, (1, 1, 80))  # (1, 80, 80) - square image
        spectrogram = torch.FloatTensor(spectrogram)

        label = torch.LongTensor([self.labels[file_id]])

        return {
            'audio_features': audio_features,
            'text_input_ids': text_input_ids,
            'text_attention_mask': text_attention_mask,
            'spectrogram': spectrogram,
            'label': label,
            'file_id': file_id
        }

class ADReSSoTrainer:
    """Training and evaluation class"""
    def __init__(self, model, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=5
        )

        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch_idx, batch in enumerate(train_loader):
            # Move to device
            audio_features = batch['audio_features'].to(self.device)
            text_input_ids = batch['text_input_ids'].to(self.device)
            text_attention_mask = batch['text_attention_mask'].to(self.device)
            spectrograms = batch['spectrogram'].to(self.device)
            labels = batch['label'].squeeze().to(self.device)

            # Forward pass
            self.optimizer.zero_grad()
            outputs = self.model(audio_features, text_input_ids, text_attention_mask, spectrograms)
            loss = self.criterion(outputs, labels)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()

            # Statistics
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            if batch_idx % 10 == 0:
                print(f'Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')

        avg_loss = total_loss / len(train_loader)
        accuracy = 100. * correct / total

        return avg_loss, accuracy

    def validate(self, val_loader):
        self.model.eval()
        total_loss = 0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        all_probs = []

        with torch.no_grad():
            for batch in val_loader:
                # Move to device
                audio_features = batch['audio_features'].to(self.device)
                text_input_ids = batch['text_input_ids'].to(self.device)
                text_attention_mask = batch['text_attention_mask'].to(self.device)
                spectrograms = batch['spectrogram'].to(self.device)
                labels = batch['label'].squeeze().to(self.device)

                # Forward pass
                outputs = self.model(audio_features, text_input_ids, text_attention_mask, spectrograms)
                loss = self.criterion(outputs, labels)

                # Statistics
                total_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                # Store for detailed metrics
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(F.softmax(outputs, dim=1).cpu().numpy())

        avg_loss = total_loss / len(val_loader)
        accuracy = 100. * correct / total

        return avg_loss, accuracy, all_preds, all_labels, all_probs

    def train(self, train_loader, val_loader, num_epochs=5):
        print(f"Training on {self.device}")
        print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")

        best_val_acc = 0
        patience_counter = 0

        for epoch in range(num_epochs):
            print(f'\nEpoch {epoch+1}/{num_epochs}')
            print('-' * 50)

            # Training
            train_loss, train_acc = self.train_epoch(train_loader)
            self.train_losses.append(train_loss)
            self.train_accuracies.append(train_acc)

            # Validation
            val_loss, val_acc, val_preds, val_labels, val_probs = self.validate(val_loader)
            self.val_losses.append(val_loss)
            self.val_accuracies.append(val_acc)

            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

            # Learning rate scheduling
            self.scheduler.step(val_loss)

            # Early stopping and model saving
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(self.model.state_dict(), 'best_adresso_model.pth')
                patience_counter = 0
                print(f'New best validation accuracy: {best_val_acc:.2f}%')
            else:
                patience_counter += 1
                if patience_counter >= 10:
                    print('Early stopping triggered')
                    break

        print(f'\nTraining completed. Best validation accuracy: {best_val_acc:.2f}%')

    def evaluate_detailed(self, test_loader, class_names=['CN', 'AD']):
        """Detailed evaluation with metrics and visualizations"""
        self.model.eval()
        all_preds = []
        all_labels = []
        all_probs = []
        all_file_ids = []

        with torch.no_grad():
            for batch in test_loader:
                # Move to device
                audio_features = batch['audio_features'].to(self.device)
                text_input_ids = batch['text_input_ids'].to(self.device)
                text_attention_mask = batch['text_attention_mask'].to(self.device)
                spectrograms = batch['spectrogram'].to(self.device)
                labels = batch['label'].squeeze().to(self.device)

                # Forward pass
                outputs = self.model(audio_features, text_input_ids, text_attention_mask, spectrograms)
                probs = F.softmax(outputs, dim=1)
                _, predicted = outputs.max(1)

                # Store results
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
                all_file_ids.extend(batch['file_id'])

        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

        # ROC AUC (for binary classification)
        if len(class_names) == 2:
            probs_positive = [prob[1] for prob in all_probs]
            auc = roc_auc_score(all_labels, probs_positive)
        else:
            auc = None

        # Confusion matrix
        cm = confusion_matrix(all_labels, all_preds)

        # Print results
        print("="*60)
        print("DETAILED EVALUATION RESULTS")
        print("="*60)
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        if auc:
            print(f"ROC AUC: {auc:.4f}")

        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=class_names, yticklabels=class_names)
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.show()

        # Plot training curves
        self.plot_training_curves()

        # Create results DataFrame
        results_df = pd.DataFrame({
            'file_id': all_file_ids,
            'true_label': all_labels,
            'predicted_label': all_preds,
            'confidence': [max(prob) for prob in all_probs],
            'prob_CN': [prob[0] for prob in all_probs],
            'prob_AD': [prob[1] for prob in all_probs] if len(all_probs[0]) > 1 else [0] * len(all_probs)
        })

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc,
            'confusion_matrix': cm,
            'results_df': results_df
        }

    def plot_training_curves(self):
        """Plot training and validation curves"""
        fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(15, 5))

        # Loss curves
        ax1.plot(self.train_losses, label='Training Loss', color='blue')
        ax1.plot(self.val_losses, label='Validation Loss', color='red')
        ax1.set_title('Training and Validation Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)

        # Accuracy curves
        ax2.plot(self.train_accuracies, label='Training Accuracy', color='blue')
        ax2.plot(self.val_accuracies, label='Validation Accuracy', color='red')
        ax2.set_title('Training and Validation Accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy (%)')
        ax2.legend()
        ax2.grid(True)

        plt.tight_layout()
        plt.show()

def visualize_semantic_graph(text_features, audio_features, file_id, save_path=None):
    """Visualize semantic relationships between audio and text"""
    # Compute similarity
    similarity = F.cosine_similarity(text_features, audio_features, dim=0).item()

    # Create networkx graph
    G = nx.Graph()
    G.add_node("Text", type="text", features=text_features[:5].tolist())
    G.add_node("Audio", type="audio", features=audio_features[:5].tolist())

    # Add edge if similarity is significant
    if similarity > 0.1:
        G.add_edge("Text", "Audio", weight=similarity, similarity=similarity)

    # Visualize
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G, k=3, iterations=50)

    # Draw nodes
    node_colors = ['lightblue' if G.nodes[node]['type'] == 'text' else 'lightcoral'
                   for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=3000, alpha=0.7)

    # Draw edges
    if G.edges():
        edge_widths = [G[u][v]['weight'] * 10 for u, v in G.edges()]
        nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.6, edge_color='gray')

    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')

    # Add edge labels
    if G.edges():
        edge_labels = {(u, v): f"Sim: {G[u][v]['similarity']:.3f}"
                      for u, v in G.edges()}
        nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=10)

    plt.title(f'Semantic Relationship Graph - {file_id}\nSimilarity: {similarity:.3f}')
    plt.axis('off')
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

    return G, similarity

# Step 6: Model Architecture Extension for ADReSSoAnalyzer
def extend_analyzer_with_model():
    """Extend the ADReSSoAnalyzer class with the new model architecture"""

    class ADReSSoAnalyzerExtended(ADReSSoAnalyzer):
        def __init__(self, base_path="/content/drive/MyDrive/Voice/extracted/ADReSSo21"):
            super().__init__(base_path)
            self.model = None
            self.trainer = None
            self.scaler = StandardScaler()

        def step_6_define_model_architecture(self):
            """Step 6: Define the multi-modal model architecture"""
            print("\n" + "="*60)
            print("STEP 6: DEFINING MODEL ARCHITECTURE")
            print("="*60)

            print("Initializing Multi-Modal Architecture:")
            print("- Graph-based Attention Module")
            print("- Vision Transformer Module")
            print("- U-Net Module")
            print("- AlexNet Module")
            print("- BERT for text processing")
            print("- Fusion and Classification layers")

            # Initialize model
            self.model = MultiModalADReSSoModel(
                audio_feature_dim=768,  # Wav2Vec2 dimension
                text_feature_dim=768,   # BERT dimension
                spectrogram_height=80,  # Mel spectrogram bins
                num_classes=2           # AD vs CN
            )

            # Initialize trainer
            self.trainer = ADReSSoTrainer(self.model)

            print(f"\nModel initialized with {sum(p.numel() for p in self.model.parameters()):,} parameters")
            print("Architecture components ready for training!")

            return self.model

        def step_7_train_model(self, features_dict, linguistic_features, batch_size=8, num_epochs=30):
            """Step 7: Train the multi-modal model"""
            print("\n" + "="*60)
            print("STEP 7: TRAINING MODEL")
            print("="*60)

            if self.model is None:
                print("Model not initialized. Running Step 6 first...")
                self.step_6_define_model_architecture()

            # Prepare labels based on file categories
            labels = {}
            for file_id in features_dict.keys():
                if 'diagnosis_ad' in file_id or 'progression_decline' in file_id:
                    labels[file_id] = 1  # AD/Decline
                else:
                    labels[file_id] = 0  # CN/No decline

            print(f"Dataset summary:")
            print(f"- Total files: {len(features_dict)}")
            print(f"- AD/Decline cases: {sum(labels.values())}")
            print(f"- CN/No decline cases: {len(labels) - sum(labels.values())}")

            # Split data
            file_ids = list(features_dict.keys())
            train_ids, test_ids = train_test_split(file_ids, test_size=0.2,
                                                 stratify=[labels[f] for f in file_ids],
                                                 random_state=42)
            train_ids, val_ids = train_test_split(train_ids, test_size=0.2,
                                                stratify=[labels[f] for f in train_ids],
                                                random_state=42)

            print(f"\nData split:")
            print(f"- Training: {len(train_ids)} files")
            print(f"- Validation: {len(val_ids)} files")
            print(f"- Testing: {len(test_ids)} files")

            # Create datasets
            train_features = {fid: features_dict[fid] for fid in train_ids}
            val_features = {fid: features_dict[fid] for fid in val_ids}
            test_features = {fid: features_dict[fid] for fid in test_ids}

            train_linguistic = {fid: linguistic_features[fid] for fid in train_ids}
            val_linguistic = {fid: linguistic_features[fid] for fid in val_ids}
            test_linguistic = {fid: linguistic_features[fid] for fid in test_ids}

            train_labels = {fid: labels[fid] for fid in train_ids}
            val_labels = {fid: labels[fid] for fid in val_ids}
            test_labels = {fid: labels[fid] for fid in test_ids}

            # Create data loaders
            train_dataset = ADReSSoDataset(train_features, train_linguistic, train_labels)
            val_dataset = ADReSSoDataset(val_features, val_linguistic, val_labels)
            test_dataset = ADReSSoDataset(test_features, test_linguistic, test_labels)

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            print(f"\nStarting training for {num_epochs} epochs...")

            # Train the model
            self.trainer.train(train_loader, val_loader, num_epochs=num_epochs)

            # Store test loader for evaluation
            self.test_loader = test_loader

            print("Training completed!")
            return self.trainer

        def step_8_evaluate_model(self, visualize_graphs=True, num_graph_samples=5):
            """Step 8: Evaluate model and visualize semantic relationships"""
            print("\n" + "="*60)
            print("STEP 8: MODEL EVALUATION AND SEMANTIC ANALYSIS")
            print("="*60)

            if self.trainer is None:
                print("Model not trained. Please run Step 7 first.")
                return None

            # Load best model
            self.model.load_state_dict(torch.load('best_adresso_model.pth'))

            # Detailed evaluation
            print("Performing detailed evaluation...")
            evaluation_results = self.trainer.evaluate_detailed(
                self.test_loader, class_names=['CN', 'AD']
            )

            # Visualize semantic relationships
            if visualize_graphs:
                print(f"\nVisualizing semantic relationships for {num_graph_samples} samples...")
                self.visualize_semantic_relationships(num_samples=num_graph_samples)

            # Additional analysis
            self.analyze_feature_importance()
            self.generate_evaluation_report(evaluation_results)

            return evaluation_results

        def visualize_semantic_relationships(self, num_samples=5):
            """Visualize semantic graphs for sample files"""
            self.model.eval()
            sample_count = 0

            with torch.no_grad():
                for batch in self.test_loader:
                    if sample_count >= num_samples:
                        break

                    # Process batch
                    audio_features = batch['audio_features'].to(self.trainer.device)
                    text_input_ids = batch['text_input_ids'].to(self.trainer.device)
                    text_attention_mask = batch['text_attention_mask'].to(self.trainer.device)
                    file_ids = batch['file_id']

                    # Get BERT features
                    bert_outputs = self.model.bert(
                        input_ids=text_input_ids,
                        attention_mask=text_attention_mask
                    )
                    text_features = bert_outputs.last_hidden_state.mean(dim=1)

                    # Visualize each sample in batch
                    for i in range(min(len(file_ids), num_samples - sample_count)):
                        file_id = file_ids[i]
                        text_feat = text_features[i]
                        audio_feat = audio_features[i]

                        print(f"\nVisualizing semantic relationships for: {file_id}")

                        # Create and visualize graph
                        graph, similarity = visualize_semantic_graph(
                            text_feat.cpu(),
                            audio_feat.cpu(),
                            file_id,
                            save_path=f"semantic_graph_{file_id}.png"
                        )

                        # Print relationship analysis
                        self.analyze_semantic_relationship(text_feat.cpu(), audio_feat.cpu(), file_id)

                        sample_count += 1

                        if sample_count >= num_samples:
                            break

        def analyze_semantic_relationship(self, text_features, audio_features, file_id):
            """Analyze the semantic relationship between audio and text"""
            # Compute various similarity metrics
            cosine_sim = F.cosine_similarity(text_features, audio_features, dim=0).item()

            # L2 distance (normalized)
            l2_distance = torch.norm(text_features - audio_features).item()
            normalized_l2 = l2_distance / (torch.norm(text_features) + torch.norm(audio_features)).item()

            # Dot product similarity
            dot_product = torch.dot(text_features, audio_features).item()

            print(f"Semantic Relationship Analysis for {file_id}:")
            print(f"  - Cosine Similarity: {cosine_sim:.4f}")
            print(f"  - Normalized L2 Distance: {normalized_l2:.4f}")
            print(f"  - Dot Product: {dot_product:.4f}")

            # Interpretation
            if cosine_sim > 0.7:
                relationship = "Strong positive correlation"
            elif cosine_sim > 0.3:
                relationship = "Moderate positive correlation"
            elif cosine_sim > 0.1:
                relationship = "Weak positive correlation"
            elif cosine_sim > -0.1:
                relationship = "No significant correlation"
            else:
                relationship = "Negative correlation"

            print(f"  - Relationship Interpretation: {relationship}")

            return {
                'cosine_similarity': cosine_sim,
                'l2_distance': normalized_l2,
                'dot_product': dot_product,
                'relationship': relationship
            }

        def analyze_feature_importance(self):
            """Analyze feature importance across different modalities"""
            print("\nAnalyzing feature importance across modalities...")

            self.model.eval()
            modality_contributions = []

            with torch.no_grad():
                for batch in self.test_loader:
                    # Get features
                    audio_features = batch['audio_features'].to(self.trainer.device)
                    text_input_ids = batch['text_input_ids'].to(self.trainer.device)
                    text_attention_mask = batch['text_attention_mask'].to(self.trainer.device)
                    spectrograms = batch['spectrogram'].to(self.trainer.device)

                    # Process text with BERT
                    bert_outputs = self.model.bert(input_ids=text_input_ids, attention_mask=text_attention_mask)
                    text_features = bert_outputs.last_hidden_state.mean(dim=1)

                    # Get individual modality outputs
                    graph_batch = self.model.create_semantic_graph(text_features, audio_features)
                    graph_out = self.model.graph_attention(graph_batch.x, graph_batch.edge_index, graph_batch.batch)
                    vit_out = self.model.vision_transformer(spectrograms)

                    audio_1d = audio_features.unsqueeze(1)
                    unet_out = self.model.unet(audio_1d)
                    alexnet_out = self.model.alexnet(audio_features)

                    # Calculate contribution magnitudes
                    contributions = {
                        'Graph Attention': torch.norm(graph_out, dim=1).mean().item(),
                        'Vision Transformer': torch.norm(vit_out, dim=1).mean().item(),
                        'U-Net': torch.norm(unet_out, dim=1).mean().item(),
                        'AlexNet': torch.norm(alexnet_out, dim=1).mean().item()
                    }

                    modality_contributions.append(contributions)

                    # Only analyze first few batches to save time
                    if len(modality_contributions) >= 5:
                        break

            # Aggregate results
            avg_contributions = {}
            for modality in modality_contributions[0].keys():
                avg_contributions[modality] = np.mean([contrib[modality] for contrib in modality_contributions])

            # Visualize feature importance
            plt.figure(figsize=(10, 6))
            modalities = list(avg_contributions.keys())
            contributions = list(avg_contributions.values())

            bars = plt.bar(modalities, contributions, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'])
            plt.title('Average Feature Contribution by Modality')
            plt.ylabel('Average L2 Norm')
            plt.xlabel('Modality')
            plt.xticks(rotation=45)

            # Add value labels on bars
            for bar, value in zip(bars, contributions):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.3f}', ha='center', va='bottom')

            plt.tight_layout()
            plt.show()

            print("Feature Importance Analysis:")
            for modality, contribution in avg_contributions.items():
                print(f"  - {modality}: {contribution:.4f}")

            return avg_contributions

        def generate_evaluation_report(self, evaluation_results):
            """Generate comprehensive evaluation report"""
            print("\n" + "="*60)
            print("COMPREHENSIVE EVALUATION REPORT")
            print("="*60)

            # Performance metrics
            print("PERFORMANCE METRICS:")
            print(f"  - Overall Accuracy: {evaluation_results['accuracy']:.4f}")
            print(f"  - Precision: {evaluation_results['precision']:.4f}")
            print(f"  - Recall: {evaluation_results['recall']:.4f}")
            print(f"  - F1-Score: {evaluation_results['f1']:.4f}")
            if evaluation_results['auc']:
                print(f"  - ROC AUC: {evaluation_results['auc']:.4f}")

            # Results by category
            results_df = evaluation_results['results_df']

            print("\nRESULTS BY CATEGORY:")
            for category in ['diagnosis_ad', 'diagnosis_cn', 'progression_decline', 'progression_no_decline']:
                category_results = results_df[results_df['file_id'].str.contains(category)]
                if len(category_results) > 0:
                    accuracy = (category_results['true_label'] == category_results['predicted_label']).mean()
                    avg_confidence = category_results['confidence'].mean()
                    print(f"  - {category}:")
                    print(f"    * Accuracy: {accuracy:.4f}")
                    print(f"    * Average Confidence: {avg_confidence:.4f}")
                    print(f"    * Sample Count: {len(category_results)}")

            # Misclassification analysis
            print("\nMISCLASSIFICATION ANALYSIS:")
            misclassified = results_df[results_df['true_label'] != results_df['predicted_label']]
            print(f"  - Total Misclassified: {len(misclassified)}")
            print(f"  - Misclassification Rate: {len(misclassified)/len(results_df):.4f}")

            if len(misclassified) > 0:
                print("  - Misclassified Samples:")
                for _, row in misclassified.head().iterrows():
                    print(f"    * {row['file_id']}: True={row['true_label']}, Pred={row['predicted_label']}, Conf={row['confidence']:.3f}")

            # High confidence predictions
            high_conf = results_df[results_df['confidence'] > 0.9]
            print(f"\nHIGH CONFIDENCE PREDICTIONS (>0.9):")
            print(f"  - Count: {len(high_conf)}")
            print(f"  - Accuracy: {(high_conf['true_label'] == high_conf['predicted_label']).mean():.4f}")

            # Save detailed results
            results_df.to_csv(f"{self.output_path}/detailed_evaluation_results.csv", index=False)

            # Save evaluation summary
            summary_dict = {
                'accuracy': evaluation_results['accuracy'],
                'precision': evaluation_results['precision'],
                'recall': evaluation_results['recall'],
                'f1_score': evaluation_results['f1'],
                'roc_auc': evaluation_results['auc'],
                'total_samples': len(results_df),
                'misclassified_count': len(misclassified),
                'high_confidence_count': len(high_conf)
            }

            with open(f"{self.output_path}/evaluation_summary.json", 'w') as f:
                json.dump(summary_dict, f, indent=2)

            print(f"\nDetailed results saved to:")
            print(f"  - {self.output_path}/detailed_evaluation_results.csv")
            print(f"  - {self.output_path}/evaluation_summary.json")

            return summary_dict

        def run_complete_pipeline_with_model(self, num_epochs=30, batch_size=8):
            """Run the complete pipeline including model training and evaluation"""
            print("="*80)
            print("COMPLETE ADReSSo MULTI-MODAL ANALYSIS PIPELINE")
            print("="*80)

            # Run original pipeline (Steps 0-5)
            results = self.run_complete_pipeline()

            if results is None:
                print("Error in initial pipeline. Cannot proceed with model training.")
                return None

            # Extract features and linguistic data
            features_dict = {}
            audio_files = results['audio_files']

            # Extract acoustic features for all files
            print("\n" + "="*60)
            print("EXTRACTING ACOUSTIC FEATURES FOR MODEL TRAINING")
            print("="*60)

            for category, files in audio_files.items():
                print(f"\nProcessing {category}...")
                for file_path in files:
                    filename = os.path.basename(file_path)
                    file_id = f"{category}_{filename}"

                    print(f"  Extracting features for {filename}...")
                    features = self.extract_acoustic_features(file_path)

                    if features is not None:
                        features_dict[file_id] = features
                    else:
                        print(f"  Warning: Could not extract features for {filename}")

            print(f"\nSuccessfully extracted features for {len(features_dict)} files")

            # Step 6: Define model architecture
            self.step_6_define_model_architecture()

            # Step 7: Train model
            self.step_7_train_model(
                features_dict,
                results['linguistic_features'],
                batch_size=batch_size,
                num_epochs=num_epochs
            )

            # Step 8: Evaluate model
            evaluation_results = self.step_8_evaluate_model()

            print("\n" + "="*80)
            print("PIPELINE COMPLETED SUCCESSFULLY!")
            print("="*80)
            print("\nAll results saved to output directory.")
            print("Model training and evaluation completed with semantic relationship analysis.")

            return {
                'original_results': results,
                'features_dict': features_dict,
                'model': self.model,
                'trainer': self.trainer,
                'evaluation_results': evaluation_results
            }

    return ADReSSoAnalyzerExtended

# Usage Example
# if __name__ == "__main__":
#     # Create extended analyzer
#     ExtendedAnalyzer = extend_analyzer_with_model()
#     analyzer = ExtendedAnalyzer()

#     # Run complete pipeline
#     print("Starting ADReSSo Multi-Modal Analysis Pipeline...")
#     complete_results = analyzer.run_complete_pipeline_with_model(
#         num_epochs=5,  # Adjust based on your computational resources
#         batch_size=4    # Adjust based on your GPU memory
#     )

#     if complete_results:
#         print("\nPipeline completed successfully!")
#         print("Check the output directory for all results and visualizations.")
#     else:
#         print("Pipeline encountered errors. Please check the dataset path and requirements.")

In [ ]:
import os
import pickle
from pathlib import Path
import numpy as np
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

# Import the necessary classes from your original file
# Adjust the file name if your original code is saved under a different name
from adress_analyzer import ADReSSoAnalyzer, extend_analyzer_with_model

class FeatureExtractionCheckpointer:
    def __init__(self, analyzer, output_dir="/content/drive/MyDrive/Voice/extracted/ADReSSo21/checkpoints"):
        """
        Initialize checkpointer for feature extraction

        Args:
            analyzer: Instance of ADReSSoAnalyzer or ADReSSoAnalyzerExtended
            output_dir: Directory to store checkpoints and features
        """
        self.analyzer = analyzer
        self.output_dir = output_dir
        self.checkpoint_dir = os.path.join(output_dir, "checkpoints")
        self.feature_dir = os.path.join(output_dir, "features")

        # Create directories if they don't exist
        os.makedirs(self.checkpoint_dir, exist_ok=True)
        os.makedirs(self.feature_dir, exist_ok=True)

        # Initialize checkpoint tracking
        self.checkpoint_file = os.path.join(self.checkpoint_dir, "checkpoint.pkl")
        self.processed_files = set()
        self.features_dict = {}

        # Load existing checkpoint if available
        self.load_checkpoint()

    def load_checkpoint(self):
        """Load existing checkpoint if available"""
        if os.path.exists(self.checkpoint_file):
            try:
                with open(self.checkpoint_file, 'rb') as f:
                    checkpoint = pickle.load(f)
                    self.processed_files = checkpoint.get('processed_files', set())
                    self.features_dict = checkpoint.get('features_dict', {})
                    print(f"Loaded checkpoint with {len(self.processed_files)} processed files")
            except Exception as e:
                print(f"Error loading checkpoint: {str(e)}")
                self.processed_files = set()
                self.features_dict = {}
        else:
            print("No checkpoint found, starting fresh")

    def save_checkpoint(self):
        """Save current state as checkpoint"""
        checkpoint = {
            'processed_files': self.processed_files,
            'features_dict': self.features_dict
        }
        try:
            with open(self.checkpoint_file, 'wb') as f:
                pickle.dump(checkpoint, f)
            print(f"Saved checkpoint to {self.checkpoint_file}")
        except Exception as e:
            print(f"Error saving checkpoint: {str(e)}")

    def save_individual_feature(self, file_id: str, features: Dict):
        """Save features for a single file"""
        feature_file = os.path.join(self.feature_dir, f"{file_id}_features.pkl")
        try:
            with open(feature_file, 'wb') as f:
                pickle.dump(features, f)
            print(f"Saved features for {file_id} to {feature_file}")
        except Exception as e:
            print(f"Error saving features for {file_id}: {str(e)}")

    def load_individual_feature(self, file_id: str) -> Dict:
        """Load features for a single file"""
        feature_file = os.path.join(self.feature_dir, f"{file_id}_features.pkl")
        if os.path.exists(feature_file):
            try:
                with open(feature_file, 'rb') as f:
                    return pickle.load(f)
            except Exception as e:
                print(f"Error loading features for {file_id}: {str(e)}")
                return None
        return None

    def extract_features_with_checkpoints(self, audio_files: Dict[str, List[str]]) -> Dict:
        """
        Extract features with checkpointing and incremental saving

        Args:
            audio_files: Dictionary of category-wise audio file paths
        Returns:
            Dictionary of extracted features
        """
        print("\n" + "="*60)
        print("EXTRACTING FEATURES WITH CHECKPOINTING")
        print("="*60)

        total_files = sum(len(files) for files in audio_files.values())
        processed_count = len(self.processed_files)
        print(f"Total files to process: {total_files}")
        print(f"Already processed: {processed_count}")

        for category, files in audio_files.items():
            print(f"\nProcessing {category} ({len(files)} files)...")

            for file_path in files:
                filename = os.path.basename(file_path)
                file_id = f"{category}_{filename}"

                # Skip if already processed
                if file_id in self.processed_files:
                    print(f"  Skipping {filename} (already processed)")
                    # Load existing features
                    features = self.load_individual_feature(file_id)
                    if features is not None:
                        self.features_dict[file_id] = features
                    continue

                print(f"  Extracting features for {filename}...")

                # Extract features
                try:
                    features = self.analyzer.extract_acoustic_features(file_path)

                    if features is not None:
                        # Store in memory
                        self.features_dict[file_id] = features
                        # Save individual feature file
                        self.save_individual_feature(file_id, features)
                        # Add to processed files
                        self.processed_files.add(file_id)
                        # Save checkpoint
                        self.save_checkpoint()
                        processed_count += 1
                        print(f"  Successfully processed {filename} ({processed_count}/{total_files})")
                    else:
                        print(f"  Warning: Failed to extract features for {filename}")

                except Exception as e:
                    print(f"  Error processing {filename}: {str(e)}")
                    continue

                # Periodic status update
                if processed_count % 10 == 0:
                    print(f"\nProgress: {processed_count}/{total_files} files processed")
                    print(f"Checkpoint saved at {processed_count} files")

        print(f"\nFeature extraction completed!")
        print(f"Processed {processed_count}/{total_files} files")
        print(f"Features saved to: {self.feature_dir}")
        print(f"Checkpoints saved to: {self.checkpoint_dir}")

        return self.features_dict

    def run_pipeline_with_checkpoints(self, num_epochs=30, batch_size=8):
        """
        Run the complete pipeline with checkpointing
        """
        print("="*80)
        print("RUNNING ADReSSo PIPELINE WITH CHECKPOINTING")
        print("="*80)

        # Run original pipeline up to feature extraction
        results = self.analyzer.run_complete_pipeline()

        if results is None:
            print("Error in initial pipeline. Cannot proceed.")
            return None

        # Extract features with checkpointing
        features_dict = self.extract_features_with_checkpoints(results['audio_files'])

        # Continue with model training and evaluation
        if isinstance(self.analyzer, ADReSSoAnalyzerExtended):
            print("\nContinuing with model training...")
            results = self.analyzer.run_complete_pipeline_with_model(
                num_epochs=num_epochs,
                batch_size=batch_size
            )
        else:
            print("\nUsing base analyzer, skipping model training")
            results['features_dict'] = features_dict

        print("\n" + "="*80)
        print("PIPELINE WITH CHECKPOINTING COMPLETED!")
        print("="*80)

        return results

# Usage Example
if __name__ == "__main__":
    # Mount Google Drive (required for Colab)
    from google.colab import drive
    drive.mount('/content/drive')

    # Install required packages
    !pip install librosa soundfile opensmile speechbrain transformers torch openai-whisper
    !pip install pandas numpy matplotlib seaborn torch-geometric

    # Initialize the extended analyzer
    ExtendedAnalyzer = extend_analyzer_with_model()
    analyzer = ExtendedAnalyzer(base_path="/content/drive/MyDrive/Voice/extracted/ADReSSo21")

    # Create checkpointer with the initialized analyzer
    checkpointer = FeatureExtractionCheckpointer(analyzer)

    # Run pipeline with checkpointing
    results = checkpointer.run_pipeline_with_checkpoints(
        num_epochs=5,
        batch_size=4
    )

    if results:
        print("\nPipeline with checkpointing completed successfully!")
        print(f"Check output directories for saved features and checkpoints:")
        print(f"- Features: {checkpointer.feature_dir}")
        print(f"- Checkpoints: {checkpointer.checkpoint_dir}")
    else:
        print("Pipeline encountered errors. Please check the logs.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 81.0MiB/s]


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loaded checkpoint with 271 processed files
RUNNING ADReSSo PIPELINE WITH CHECKPOINTING
=== ADReSSo21 Speech Analysis Pipeline ===

Step 0: Getting audio files...
Found 271 audio files across all categories
  diagnosis_ad: 87 files
  diagnosis_cn: 79 files
  progression_decline: 15 files
  progression_no_decline: 58 files
  progression_test: 32 files

Step 1: Demonstrating acoustic features...

Showing features for diagnosis_ad:
=== Acoustic Features for adrso047.wav ===

1. eGeMAPS Features: 88 features
   Shape: (88,)
   Sample values: [25.866777    0.31971478 16.288124   26.47413    34.226044  ]

2. MFCC Features:
   Mean: (13,) - [-283.20312     76.72155      4.9237657   23.371506   -11.170574 ]
   Std: (13,) - [79.158615 30.192682 25.279158 24.67176  18.788862]
   Delta: (13,) - [ 0.3264354  -0.00845834 -0.01544069  0.0086811  -0.01617551]
   Delta-Delta: (13,) - [ 0.01663556  0.00127252  0.00217276 -0.00220479  0.00038056]

3. Log-Mel Spectrogram Features:
   Mean: (80,) - [-23.60

NameError: name 'ADReSSoAnalyzerExtended' is not defined

In [ ]:
import os
import pickle
from pathlib import Path
import numpy as np
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

# Import the necessary components from your original file
# Adjust the file name if your original code is saved under a different name
from adress_analyzer import ADReSSoAnalyzer, extend_analyzer_with_model

class FeatureExtractionCheckpointer:
    def __init__(self, analyzer, output_dir="/content/drive/MyDrive/Voice/extracted/ADReSSo21/checkpoints"):
        """
        Initialize checkpointer for feature extraction

        Args:
            analyzer: Instance of ADReSSoAnalyzer or the extended analyzer class
            output_dir: Directory to store checkpoints and features
        """
        self.analyzer = analyzer
        self.output_dir = output_dir
        self.checkpoint_dir = os.path.join(output_dir, "checkpoints")
        self.feature_dir = os.path.join(output_dir, "features")

        # Create directories if they don't exist
        os.makedirs(self.checkpoint_dir, exist_ok=True)
        os.makedirs(self.feature_dir, exist_ok=True)

        # Initialize checkpoint tracking
        self.checkpoint_file = os.path.join(self.checkpoint_dir, "checkpoint.pkl")
        self.processed_files = set()
        self.features_dict = {}

        # Load existing checkpoint if available
        self.load_checkpoint()

    def load_checkpoint(self):
        """Load existing checkpoint if available"""
        if os.path.exists(self.checkpoint_file):
            try:
                with open(self.checkpoint_file, 'rb') as f:
                    checkpoint = pickle.load(f)
                    self.processed_files = checkpoint.get('processed_files', set())
                    self.features_dict = checkpoint.get('features_dict', {})
                    print(f"Loaded checkpoint with {len(self.processed_files)} processed files")
            except Exception as e:
                print(f"Error loading checkpoint: {str(e)}")
                self.processed_files = set()
                self.features_dict = {}
        else:
            print("No checkpoint found, starting fresh")

    def save_checkpoint(self):
        """Save current state as checkpoint"""
        checkpoint = {
            'processed_files': self.processed_files,
            'features_dict': self.features_dict
        }
        try:
            with open(self.checkpoint_file, 'wb') as f:
                pickle.dump(checkpoint, f)
            print(f"Saved checkpoint to {self.checkpoint_file}")
        except Exception as e:
            print(f"Error saving checkpoint: {str(e)}")

    def save_individual_feature(self, file_id: str, features: Dict):
        """Save features for a single file"""
        feature_file = os.path.join(self.feature_dir, f"{file_id}_features.pkl")
        try:
            with open(feature_file, 'wb') as f:
                pickle.dump(features, f)
            print(f"Saved features for {file_id} to {feature_file}")
        except Exception as e:
            print(f"Error saving features for {file_id}: {str(e)}")

    def load_individual_feature(self, file_id: str) -> Dict:
        """Load features for a single file"""
        feature_file = os.path.join(self.feature_dir, f"{file_id}_features.pkl")
        if os.path.exists(feature_file):
            try:
                with open(feature_file, 'rb') as f:
                    return pickle.load(f)
            except Exception as e:
                print(f"Error loading features for {file_id}: {str(e)}")
                return None
        return None

    def extract_features_with_checkpoints(self, audio_files: Dict[str, List[str]]) -> Dict:
        """
        Extract features with checkpointing and incremental saving

        Args:
            audio_files: Dictionary of category-wise audio file paths
        Returns:
            Dictionary of extracted features
        """
        print("\n" + "="*60)
        print("EXTRACTING FEATURES WITH CHECKPOINTING")
        print("="*60)

        total_files = sum(len(files) for files in audio_files.values())
        processed_count = len(self.processed_files)
        print(f"Total files to process: {total_files}")
        print(f"Already processed: {processed_count}")

        for category, files in audio_files.items():
            print(f"\nProcessing {category} ({len(files)} files)...")

            for file_path in files:
                filename = os.path.basename(file_path)
                file_id = f"{category}_{filename}"

                # Skip if already processed
                if file_id in self.processed_files:
                    print(f"  Skipping {filename} (already processed)")
                    # Load existing features
                    features = self.load_individual_feature(file_id)
                    if features is not None:
                        self.features_dict[file_id] = features
                    continue

                print(f"  Extracting features for {filename}...")

                # Extract features
                try:
                    features = self.analyzer.extract_acoustic_features(file_path)

                    if features is not None:
                        # Store in memory
                        self.features_dict[file_id] = features
                        # Save individual feature file
                        self.save_individual_feature(file_id, features)
                        # Add to processed files
                        self.processed_files.add(file_id)
                        # Save checkpoint
                        self.save_checkpoint()
                        processed_count += 1
                        print(f"  Successfully processed {filename} ({processed_count}/{total_files})")
                    else:
                        print(f"  Warning: Failed to extract features for {filename}")

                except Exception as e:
                    print(f"  Error processing {filename}: {str(e)}")
                    continue

                # Periodic status update
                if processed_count % 10 == 0:
                    print(f"\nProgress: {processed_count}/{total_files} files processed")
                    print(f"Checkpoint saved at {processed_count} files")

        print(f"\nFeature extraction completed!")
        print(f"Processed {processed_count}/{total_files} files")
        print(f"Features saved to: {self.feature_dir}")
        print(f"Checkpoints saved to: {self.checkpoint_dir}")

        return self.features_dict

    def run_pipeline_with_checkpoints(self, num_epochs=30, batch_size=8):
        """
        Run the complete pipeline with checkpointing
        """
        print("="*80)
        print("RUNNING ADReSSo PIPELINE WITH CHECKPOINTING")
        print("="*80)

        # Run original pipeline up to feature extraction
        results = self.analyzer.run_complete_pipeline()

        if results is None:
            print("Error in initial pipeline. Cannot proceed.")
            return None

        # Extract features with checkpointing
        features_dict = self.extract_features_with_checkpoints(results['audio_files'])

        # Continue with model training and evaluation if extended analyzer
        if hasattr(self.analyzer, 'run_complete_pipeline_with_model'):
            print("\nContinuing with model training...")
            results = self.analyzer.run_complete_pipeline_with_model(
                num_epochs=num_epochs,
                batch_size=batch_size
            )
        else:
            print("\nUsing base analyzer, skipping model training")
            results['features_dict'] = features_dict

        print("\n" + "="*80)
        print("PIPELINE WITH CHECKPOINTING COMPLETED!")
        print("="*80)

        return results

# Usage Example
if __name__ == "__main__":
    # Mount Google Drive (required for Colab)
    try:
        from google.colab import drive
        drive.mount('/content/drive')
    except ImportError:
        print("Not running in Colab, skipping drive mount")

    # Install required packages
    try:
        import subprocess
        packages = [
            "librosa", "soundfile", "opensmile", "speechbrain",
            "transformers", "torch", "openai-whisper",
            "pandas", "numpy", "matplotlib", "seaborn", "torch-geometric"
        ]
        for pkg in packages:
            subprocess.check_call(["pip", "install", pkg])
        print("All required packages installed successfully")
    except Exception as e:
        print(f"Error installing packages: {str(e)}")

    # Initialize the extended analyzer
    ExtendedAnalyzer = extend_analyzer_with_model()
    analyzer = ExtendedAnalyzer(base_path="/content/drive/MyDrive/Voice/extracted/ADReSSo21")

    # Create checkpointer with the initialized analyzer
    checkpointer = FeatureExtractionCheckpointer(analyzer)

    # Run pipeline with checkpointing
    results = checkpointer.run_pipeline_with_checkpoints(
        num_epochs=5,
        batch_size=4
    )

    if results:
        print("\nPipeline with checkpointing completed successfully!")
        print(f"Check output directories for saved features and checkpoints:")
        print(f"- Features: {checkpointer.feature_dir}")
        print(f"- Checkpoints: {checkpointer.checkpoint_dir}")
    else:
        print("Pipeline encountered errors. Please check the logs.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All required packages installed successfully


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded checkpoint with 271 processed files
RUNNING ADReSSo PIPELINE WITH CHECKPOINTING
=== ADReSSo21 Speech Analysis Pipeline ===

Step 0: Getting audio files...
Found 271 audio files across all categories
  diagnosis_ad: 87 files
  diagnosis_cn: 79 files
  progression_decline: 15 files
  progression_no_decline: 58 files
  progression_test: 32 files

Step 1: Demonstrating acoustic features...

Showing features for diagnosis_ad:
=== Acoustic Features for adrso047.wav ===

1. eGeMAPS Features: 88 features
   Shape: (88,)
   Sample values: [25.866777    0.31971478 16.288124   26.47413    34.226044  ]

2. MFCC Features:
   Mean: (13,) - [-283.20312     76.72155      4.9237657   23.371506   -11.170574 ]
   Std: (13,) - [79.158615 30.192682 25.279158 24.67176  18.788862]
   Delta: (13,) - [ 0.3264354  -0.00845834 -0.01544069  0.0086811  -0.01617551]
   Delta-Delta: (13,) - [ 0.01663556  0.00127252  0.00217276 -0.00220479  0.00038056]

3. Log-Mel Spectrogram Features:
   Mean: (80,) - [-23.60

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
def run_pipeline_with_checkpoints(self, num_epochs=30, batch_size=8):
    print("="*80)
    print("RUNNING ADReSSo PIPELINE WITH CHECKPOINTING")
    print("="*80)

    # Run original pipeline up to feature extraction
    import time
    start_time = time.time()
    results = self.analyzer.run_complete_pipeline()
    print(f"Initial pipeline took {time.time() - start_time:.2f} seconds")

    if results is None:
        print("Error in initial pipeline. Cannot proceed.")
        return None

    # Extract features with checkpointing
    start_time = time.time()
    features_dict = self.extract_features_with_checkpoints(results['audio_files'])
    print(f"Feature extraction took {time.time() - start_time:.2f} seconds")

    # Continue with model training and evaluation if extended analyzer
    if hasattr(self.analyzer, 'run_complete_pipeline_with_model'):
        print("\nContinuing with model training...")
        start_time = time.time()
        results = self.analyzer.run_complete_pipeline_with_model(
            num_epochs=num_epochs,
            batch_size=batch_size
        )
        print(f"Model training took {time.time() - start_time:.2f} seconds")
    else:
        print("\nUsing base analyzer, skipping model training")
        results['features_dict'] = features_dict

    print("\n" + "="*80)
    print("PIPELINE WITH CHECKPOINTING COMPLETED!")
    print("="*80)

    return results